In [44]:
# pip install openai==1.40.1

from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import json

In [10]:
# put your openai key here
openai_key = '...'
model = 'gpt-4o-2024-08-06'

## This is where we collect all the intial information

In [17]:
client = OpenAI(api_key= openai_key)

# this initialises the chatbot memeory
history = ''

# this is the instruction that is persistently passed to the model 
system = """You are a helpful assistant with the only role to help a human create a good schedule for their business. \n
     You role is to guide the human through the following points so that they can generate a good schedule, step by step, one question at the time: \n
     0. Introduce yourself as Lumera Scheduler assistant and greet the peroson with their name [Alessandro].
     1. What is the business name?\n
     2. What type of business is it?\n
     3. Where is the business located? In case of multiple branches in the same city, please provide a unique location identifier, such as the neighborrhood or street.\n
     4. What days do you want the business to be opened?\n
     5. What times do you want the business to be opened?\n
     6. Specify all the roles that the business need to have covered in order to function well (e.g., cook, receptionist, waiter)?\n
     7. Once all the above information has been provieded, output a summary of this conversation, containing all the details, so that another AI model can use it to generate a good schedule for this business."""

# this is the chat function, takes our system instruction, user message and chatbot memory
def chat(system, user, history):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": f'{history + user}'},
        ],
        # response_format=Schema,    # this is where you can define any schema for the output
    )

    message = completion.choices[0].message
    return message

In [24]:
# chat with the model here, just updating the "user" message as you go through the conversation
user = 'I am thinking of  a cashier, a waiter, a waitress and a cook '

# this shouldn't require touching, we just get the response and update the chatbot memory / conversation as we move through the conversation
response = chat(system, user, history)
history += (f'Human message: {user}\n')
history += (f'\nAI message: {response.content}\n')
print(history) 

Human message: Hello!

AI message: Hello! I am the Lumera Scheduler assistant. It's nice to meet you, Alessandro. How can I assist you today in creating a schedule for your business? Let's start with the business name.
Human message: Crazy Pizza!

AI message: Great, Alessandro! Crazy Pizza sounds like an exciting business. Now, can you tell me what type of business Crazy Pizza is?
Human message: Restaurant Pizzeria!

AI message: Wonderful! Crazy Pizza is a Restaurant Pizzeria. Next, could you please let me know where the business is located? If there are multiple branches in the same city, please provide a unique location identifier, such as the neighborhood or street.
Human message: We are in London

AI message: Great! Crazy Pizza is located in London. Now, could you please tell me the specific days you want Crazy Pizza to be open?
Human message: Every day

AI message: AI message: Got it, Alessandro! Crazy Pizza will be open every day. What times do you want your business to be open e

Problem: this last step, where we generate the summary, maybe could be handled by another bot specifically prompet to return a strcutured format?

## Here we create the schedule

In [31]:
scheduler_system = "You are a helpful assistant. Based on the provided information you will return a structured representation of all the given buisnes attributes, along with a schedule where roles are assigned per each day and return it in a json format."

In [67]:
class Business(BaseModel):
    id: str = Field(..., description="Unique identifier for the business.")
    name: str = Field(..., description="Name of the business.")
    email: str = Field(..., description="Contact email of the business.")
    locationId: str = Field(..., description="Identifier for the primary location of the business.")

class Location(BaseModel):
    id: str = Field(..., description="Unique identifier for the location.")
    businessId: str = Field(..., description="Identifier for the business that owns the location.")
    name: str = Field(..., description="Name of the location.")
    address: str = Field(..., description="Physical address of the location.")
    email: str = Field(..., description="Contact email for the location.")

class LocationRole(BaseModel):
    id: str = Field(..., description="Unique identifier for the location role.")
    locationId: str = Field(..., description="Identifier for the location associated with the role.")
    roleId: str = Field(..., description="Identifier for the role within the location.")

class LocationOpeningHour(BaseModel):
    id: str = Field(..., description="Unique identifier for the location opening hour entry.")
    locationId: str = Field(..., description="Identifier for the location associated with the opening hours.")
    dayOfWeek: int = Field(..., description="Day of the week as an integer (1 for Monday, 7 for Sunday).")
    openTime: str = Field(..., description="Opening time of the location on the specified day.")
    closeTime: str = Field(..., description="Closing time of the location on the specified day.")

class Role(BaseModel):
    id: str = Field(..., description="Unique identifier for the role.")
    name: str = Field(..., description="Name of the role.")
    description: str = Field(..., description="Detailed description of the role.")

class StaffRequirement(BaseModel):
    id: str = Field(..., description="Unique identifier for the staff requirement.")
    shiftId: str = Field(..., description="Identifier for the shift associated with the staff requirement.")
    roleId: str = Field(..., description="Identifier for the role required for the shift.")
    employeesRequired: int = Field(..., description="Number of employees required for the role in the shift.")

class Shift(BaseModel):
    id: str = Field(..., description="Unique identifier for the shift.")
    shiftName: str = Field(..., description="Name or label for the shift.")
    locationId: str = Field(..., description="Identifier for the location where the shift takes place.")
    startTime: str = Field(..., description="Start time of the shift.")
    endTime: str = Field(..., description="End time of the shift.")
    date: str = Field(..., description="Date of the shift.")

class Input(BaseModel):
    business: Business
    locations: List[Location]
    locationRoles: List[LocationRole]
    locationOpeningHours: List[LocationOpeningHour]
    roles: List[Role]
    staffRequirements: List[StaffRequirement]
    shifts: List[Shift]

class BusinessSchedule(BaseModel):
    input: Input

In [68]:
schema = BusinessSchedule.schema()
print(schema)

{'$defs': {'Business': {'properties': {'id': {'description': 'Unique identifier for the business.', 'title': 'Id', 'type': 'string'}, 'name': {'description': 'Name of the business.', 'title': 'Name', 'type': 'string'}, 'email': {'description': 'Contact email of the business.', 'title': 'Email', 'type': 'string'}, 'locationId': {'description': 'Identifier for the primary location of the business.', 'title': 'Locationid', 'type': 'string'}}, 'required': ['id', 'name', 'email', 'locationId'], 'title': 'Business', 'type': 'object'}, 'Input': {'properties': {'business': {'$ref': '#/$defs/Business'}, 'locations': {'items': {'$ref': '#/$defs/Location'}, 'title': 'Locations', 'type': 'array'}, 'locationRoles': {'items': {'$ref': '#/$defs/LocationRole'}, 'title': 'Locationroles', 'type': 'array'}, 'locationOpeningHours': {'items': {'$ref': '#/$defs/LocationOpeningHour'}, 'title': 'Locationopeninghours', 'type': 'array'}, 'roles': {'items': {'$ref': '#/$defs/Role'}, 'title': 'Roles', 'type': 'ar

In [69]:
def scheduler(scheduler_system, user, history):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": scheduler_system},
            {"role": "user", "content": f'{history + user}'},
        ],
        response_format=BusinessSchedule,  
    )

    message = completion.choices[0].message
    return message

In [70]:
user = """Based on the provided information, collected during a conversation with our client, provide me with a schedule where roles are assigned per each day, making sure the business is well covered at all times.\n """

In [71]:
scheduler_response = scheduler(scheduler_system, user, history)

In [56]:
parsed_json = json.loads(scheduler_response.content)
# Pretty-print the JSON with indentation
formatted_json = json.dumps(parsed_json, indent=4)
print(formatted_json)

{
    "input": {
        "business": {
            "id": "1",
            "name": "Crazy Pizza",
            "email": "contact@crazypizza.com",
            "locationId": "1"
        },
        "locations": [
            {
                "id": "1",
                "businessId": "1",
                "name": "Crazy Pizza London",
                "address": "London, UK",
                "email": "london@crazypizza.com"
            }
        ],
        "locationRoles": [
            {
                "id": "1",
                "locationId": "1",
                "roleId": "1"
            },
            {
                "id": "2",
                "locationId": "1",
                "roleId": "2"
            },
            {
                "id": "3",
                "locationId": "1",
                "roleId": "3"
            },
            {
                "id": "4",
                "locationId": "1",
                "roleId": "4"
            }
        ],
        "locationOpeningHours": [
